In [ ]:
import pandas as pd
df = pd.read_csv("input.csv.zip")

df_ml = df.copy()

df_ml["Forecasttidspunkt"] = pd.to_datetime(df_ml["Forecasttidspunkt"])
df_ml["periode_regnskabstal"] = pd.to_datetime(df_ml["periode_regnskabstal"])

df_ml.set_index("Forecasttidspunkt",inplace=True)

list_drop = ["SHROUT","company_name","CUSIP","analyst_EPS_mean","analyst_high","analyst_low","analyst_std_mean","periode_regnskabstal","PERMNO","date","slutmåned"]

df_ml.drop(list_drop,axis=1,inplace=True)



df_ml["month"] = df_ml.index.month
df_ml["year"] = df_ml.index.year

df_ml = df_ml.astype({'GVKEY': 'category'})
df_ml = df_ml.astype({'industry_fama': 'category'})

df_ml.sort_index(inplace=True)



"""# New Feautures"""

# Volume i USD
df_ml["volume_usd"] = df_ml["ALTPRC"] * df_ml["VOL"]

# Dividend Yield
df_ml["Div_yield"] = df_ml["DPS_ex_date(dvpsxq)"] / df_ml["ALTPRC"]




# Dropper duplikat kolonner

df_ml.drop(["PRC","adj_close"],axis=1,inplace=True)

# Eksporter til Datarobot

#df_ml.to_csv("/content/drive/MyDrive/Cand.fælles/Speciale/datarobot_input/baseline_model.csv.zip")

"""## Laging Variables"""

# Dropper kolonner:

columns_drop_list = ["Assets_total(Atq)","merafkast"]
df_ml.drop(columns_drop_list,axis=1,inplace=True)

columns_lag = ['EPS_actual', 'Accounts_payable(Apq)',
       'Assets_total(Atq)', 'COGS(Cogsq)',
       'Common_equity(Ceqq)', 'DPS_ex_date(dvpsxq)', 'Deprect_and_ammor(Dpq)',
       'EPS_lagged', 'GVKEY', 'Income taxes_total(Txtq)',
       'Non_operating_income(Nopiq)', 'Operating_income_after_deprec(Oiadpq)',
       'Opex_total(Xoprq)', 'PP&E_total_net(Ppentq)','PRC',
       'Pretax_income(Piq)', 'Receviables(Rectq)', 'Revenue_total(Revtq)',
    'VOL', 'adj_return','adj_close','market_cap', 'niq', 'industry_return', 'merafkast', 'ALTPRC','volume_usd','Div_yield']

df_lag = df_ml.copy()
df_lag = df_lag[df_lag.columns.intersection(columns_lag)]

df_lag_new = df_lag.copy()

for column in df_lag.columns:
  for i in range(1,6):
    df_lag_new[f'{column}_{i}'] = df_lag.groupby("GVKEY")[f'{column}'].shift(i).values

'''columns_to_choose = ['Accounts_payable(Apq)',
       'Assets_total(Atq)', 'COGS(Cogsq)',
       'Common_equity(Ceqq)', 'DPS_ex_date(dvpsxq)', 'Deprect_and_ammor(Dpq)',
       'EPS_lagged', 'Income taxes_total(Txtq)',
       'Non_operating_income(Nopiq)', 'Operating_income_after_deprec(Oiadpq)',
       'Opex_total(Xoprq)', 'PP&E_total_net(Ppentq)', 'PRC',
       'Pretax_income(Piq)', 'Receviables(Rectq)', 'Revenue_total(Revtq)',
    'VOL', 'adj_close', 'adj_return','market_cap', 'niq', 'industry_return', 'merafkast', 'ALTPRC',"volume_usd","Div_yield"]'''

columns_to_choose = ['Assets_total(Atq)','Accounts_payable(Apq)', 'COGS(Cogsq)',
       'Common_equity(Ceqq)', 'DPS_ex_date(dvpsxq)', 'Deprect_and_ammor(Dpq)',
       'EPS_lagged', 'Income taxes_total(Txtq)',
       'Non_operating_income(Nopiq)', 'Operating_income_after_deprec(Oiadpq)',
       'Opex_total(Xoprq)', 'PP&E_total_net(Ppentq)',
       'Pretax_income(Piq)', 'Receviables(Rectq)', 'Revenue_total(Revtq)',
       'market_cap', 'niq',"Div_yield","ALTPRC",'adj_return',"industry_return","merafkast",'VOL','volume_usd','PRC','adj_close']

col_lag_choose = []
for element in columns_to_choose:
  for i in range(1,6):
    col_lag_choose.append(f'{element}_{i}')
col_lag_choose.append("GVKEY")

col_lag_choose

df_lag_new = df_lag_new[df_lag_new.columns.intersection(col_lag_choose)]


df_ml = df_ml.reset_index().merge(df_lag_new.reset_index(),how="inner",on=["GVKEY","Forecasttidspunkt"])



"""# Running ML Model"""

df_ml.info(verbose=True)

numeric_columns = df_ml.select_dtypes(include=['number']).columns

# fill -1 to all NaN 
df_ml[numeric_columns] = df_ml[numeric_columns].fillna(0)

df_ml.set_index("Forecasttidspunkt",inplace=True)

cutoff_date = "2016-06-30"
validation_start = "2016-09-30"
validation_end = "2018-06-30"
test_start = "2018-09-30"

training_df = df_ml.loc[:cutoff_date,:]
validation_df = df_ml.loc[validation_start:validation_end,:]
test_df = df_ml.loc[test_start:,:]

training_df_cv =  df_ml.loc[:validation_end,:]

# X og Y

training_y = training_df["EPS_actual"]
validatation_y = validation_df["EPS_actual"]
test_y = test_df["EPS_actual"]
training_y_cv = training_df_cv["EPS_actual"]

training_x = training_df.drop("EPS_actual",axis=1)
training_x_cv = training_df_cv.drop("EPS_actual",axis=1)
validatation_x = validation_df.drop("EPS_actual",axis=1)
test_x = test_df.drop("EPS_actual",axis=1)


In [ ]:
df_ml[df_ml.industry_fama=="Business Equipment"].to_csv('buseq_azure.csv')

In [ ]:
df_ml.to_csv("csvlokalazure.csv")

In [ ]:
from catboost import CatBoostRegressor
from catboost import Pool

cat_model = CatBoostRegressor(cat_features=["GVKEY","industry_fama"],random_seed=2021)

pool_val = Pool(validatation_x,validatation_y,cat_features=["GVKEY","industry_fama"])

cat_model.fit(training_x,training_y,eval_set=pool_val)

In [ ]:
validation_pred = cat_model.predict(validatation_x)

from sklearn.metrics import mean_squared_error

val_mse = mean_squared_error(validatation_y,validation_pred,squared=False)

val_mse
